In [1]:
import sys
sys.path.append('.')  

import pyro
import torch

from data.mushroom_data import load_mushroom_data

from model.pyrobayesian1dcnn import PyroBayesian1DCNN
from train.train import train_model
from train.evaluate import evaluate_model
from train.hyperparam_opt import run_hyperparam_optimization

from warnings import filterwarnings
filterwarnings('ignore')

pyro.set_rng_seed(42)
torch.manual_seed(42)

torch.cuda.is_available()

/home/emmanuel/miniconda3/envs/deep_bayesian_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
train_loader, test_loader, input_dim = load_mushroom_data(
    batch_size=64,
    test_size=0.2,
    random_state=42
)

model = PyroBayesian1DCNN(
    input_dim=input_dim,
    output_dim=1,
    proj_channels=16,
    proj_seq_len=4,
    conv_out_channels=32,
    kernel_size=3,
    prior_std=0.1
)

svi = train_model(
    model=model,
    train_loader=train_loader,
    num_epochs=1,
    lr=0.001,
    num_particles=1
)

f1_simple = evaluate_model(model, test_loader, num_samples=10)
print(f"f1 (simple config) = {f1_simple*100:.2f}%")

study = run_hyperparam_optimization(n_trials=10) 

best_params = study.best_params
print("Best hyperparams found by Optuna:", best_params)

Utilisation de : cpu
tab2seq.linear.weight: cpu
tab2seq.linear.bias: cpu
bn1.weight: cpu
bn1.bias: cpu
bn2.weight: cpu
bn2.bias: cpu
[Epoch 1] Loss: 0.7491


[I 2025-01-21 15:05:41,750] A new study created in memory with name: no-name-2dcdd88f-2a0d-4661-90e8-40f329921e7b


f1 (simple config) = 52.84%
Utilisation de : cpu
tab2seq.linear.weight: cpu
tab2seq.linear.bias: cpu
bn1.weight: cpu
bn1.bias: cpu
bn2.weight: cpu
bn2.bias: cpu
[Epoch 1] Loss: 0.7720
[Epoch 2] Loss: 0.7707
[Epoch 3] Loss: 0.7678
[Epoch 4] Loss: 0.7747
[Epoch 5] Loss: 0.7650


In [ ]:
model_optimized = PyroBayesian1DCNN(
    input_dim=input_dim,
    output_dim=1,
    proj_channels=best_params["proj_channels"],
    proj_seq_len=best_params["proj_seq_len"],
    conv_out_channels=best_params["conv_out_channels"],
    kernel_size=best_params["kernel_size"],
    prior_std=best_params["prior_std"]
)

pyro.set_rng_seed(42)
torch.manual_seed(42)

svi_opt = train_model(
    model=model_optimized,
    train_loader=train_loader,
    num_epochs=best_params["num_epochs"],
    lr=best_params["lr"],
    num_particles=1
)

f1_opt = evaluate_model(model_optimized, test_loader, num_samples=10)
print(f"F1 (optimized) = {f1_opt*100:.2f}%")

[Epoch 1] Loss: 1.0407
[Epoch 2] Loss: 1.0496
[Epoch 3] Loss: 1.0548
[Epoch 4] Loss: 1.0298
[Epoch 5] Loss: 1.0387
[Epoch 6] Loss: 1.0277
[Epoch 7] Loss: 1.0505
[Epoch 8] Loss: 1.0514
[Epoch 9] Loss: 1.0542
[Epoch 10] Loss: 1.0396
Accuracy (optimized) = 52.66%
